In [1]:
import requests as req
import pandas as pd
import yfinance as yf

import psycopg2
from psycopg2.extras import execute_batch, RealDictCursor

from sqlalchemy import create_engine


In [2]:
### DB CONNECTION ###
DATABASE_NAME = "postgres"
HOSTNAME = "localhost"
USER = "postgres"
PASSWORD = "test1234"
PORT="5432"


db_connection = psycopg2.connect(
            database = DATABASE_NAME,
            host = HOSTNAME,
            user = USER,
            password = PASSWORD,
            port = PORT
        )

engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOSTNAME}:{PORT}/{DATABASE_NAME}')

In [3]:
## CREATE SCHEMA
schema = "project_db"

create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {schema};"


## CREATE TABLE Stock_Prices_Tbl
create_table_stock_query = f"""CREATE TABLE IF NOT EXISTS {schema}."Stock_Prices_Tbl" (
                                    "Stock_ID" VARCHAR(100),
                                    "Date" DATE,
                                    "Open_Price" DOUBLE PRECISION,
                                    "Close_Price" DOUBLE PRECISION,
                                    "Low" DOUBLE PRECISION,
                                    "High" DOUBLE PRECISION,
                                    "Volume_traded" INTEGER,
                                    CONSTRAINT pk_stock_prices PRIMARY KEY ("Stock_ID", "Date")
    );
    """

## CREATE TABLE Industry_Tbl
create_table_industry_query = f"""CREATE TABLE IF NOT EXISTS {schema}."Industry_Tbl" (
                                    "Activity_ID" SERIAL,
                                    "Activity_Name" VARCHAR(100),
                                    "Industry" VARCHAR(100),
                                    CONSTRAINT pk_industry PRIMARY KEY ("Activity_ID")
    );
    """

## CREATE TABLE Index_Tbl    
create_table_index_query = f"""CREATE TABLE IF NOT EXISTS {schema}."Index_Tbl" (
                                    "Index_ID" VARCHAR(100),
                                    "Index_Name" VARCHAR(100),
                                    CONSTRAINT pk_index PRIMARY KEY ("Index_ID")
    );
    """

## CREATE TABLE Country_Tbl
create_table_country_query = f"""CREATE TABLE IF NOT EXISTS {schema}."Country_Tbl" (
                                    "Country_ID" SERIAL,
                                    "Country_Name" VARCHAR(100),
                                    "Economic_Region" VARCHAR(300),
                                    CONSTRAINT pk_country PRIMARY KEY ("Country_ID")
    );
    """

## CREATE TABLE Company_Tbl    
create_table_company_query = f"""CREATE TABLE IF NOT EXISTS {schema}."Company_Tbl" (
                                    "Company_ID" VARCHAR(100),
                                    "Company_Name" VARCHAR(200),
                                    "Activity_ID" INTEGER,
                                    "Index_ID" VARCHAR(100),
                                    "Country_Headquarter" INTEGER,
                                    CONSTRAINT pk_company PRIMARY KEY ("Company_ID"),
                                    CONSTRAINT fk_activity_id FOREIGN KEY ("Activity_ID") REFERENCES {schema}."Industry_Tbl" ("Activity_ID") ON DELETE SET NULL,
                                    CONSTRAINT fk_index_id FOREIGN KEY ("Index_ID") REFERENCES {schema}."Index_Tbl" ("Index_ID") ON DELETE SET NULL,
                                    CONSTRAINT fk_country_headquarter FOREIGN KEY ("Country_Headquarter") REFERENCES {schema}."Country_Tbl" ("Country_ID") ON DELETE SET NULL
    );
    """

## Execute Statements    
with db_connection.cursor(cursor_factory=RealDictCursor) as cur:
    cur.execute(create_schema_query)
    cur.execute(create_table_stock_query)
    cur.execute(create_table_industry_query)
    cur.execute(create_table_country_query)
    cur.execute(create_table_index_query)
    cur.execute(create_table_company_query)
    cur.connection.commit()

In [4]:
ticker_index = "^STOXX50E"

euro_stoxx_50_tickers = [
    "ABI.BR",    # Anheuser-Busch InBev SA/NV (Belgien)
    "AI.PA",     # Air Liquide S.A. (Frankreich)
    "AIR.PA",    # Airbus SE (Frankreich)
    "ALV.DE",    # Allianz SE (Deutschland)
    "ASML.AS",   # ASML Holding N.V. (Niederlande)
    "BAS.DE",    # BASF SE (Deutschland)
    "BBVA.MC",   # Banco Bilbao Vizcaya Argentaria, S.A. (Spanien)
    "BAYN.DE",   # Bayer AG (Deutschland)
    "BN.PA",     # Danone S.A. (Frankreich)
    "CRG.IR",    # CRH plc (Irland)
    "CS.PA",     # AXA S.A. (früher CS war AXA; ggf. AXA.PA)
    "DAI.DE",    # Mercedes-Benz Group AG (Deutschland)
    "DB1.DE",    # Deutsche Boerse AG (Deutschland)
    "DG.PA",     # VINCI S.A. (Frankreich) – (ehem. DG)
    "DPW.DE",    # Deutsche Post AG (Deutschland)
    "DTE.DE",    # Deutsche Telekom AG (Deutschland)
    "EN.PA",     # Bouygues S.A. oder (ENGI.PA für Engie S.A.)
    "ENG.MC",    # Enagas S.A. (Spanien) – Achtung: Je nach Indexzusammensetzung
    "ENEL.MI",   # Enel S.p.A. (Italien)
    "ENEI.MI",   # Eni S.p.A. (Italien)
    "EONGY",     # E.ON SE (evtl. EOA.DE, je nach Handelsplatz)
    "FER.MC",    # Ferrovial S.A. (Spanien)
    "GRF.PA",    # Groupe Renault S.A. (Frankreich) – evtl. RNO.PA
    "IBE.MC",    # Iberdrola S.A. (Spanien)
    "INA.MI",    # Intesa Sanpaolo S.p.A. (Italien)
    "ITX.MC",    # Industria de Diseno Textil, S.A. (Inditex) (Spanien)
    "KER.PA",    # Kering S.A. (Frankreich)
    "MC.PA",     # LVMH Moet Hennessy Louis Vuitton SE (Frankreich)
    "MUV2.DE",   # Muenchener Rueckversicherungs-Gesellschaft AG (Münchener Rück)
    "NG.PA",     # BUREAU VERITAS oder (OR.PA für L'Oreal) – hier muss man aufpassen
    "OR.PA",     # L'Oreal S.A. (Frankreich)
    "PHIA.AS",   # Koninklijke Philips N.V. (Niederlande)
    "RACE.MI",   # Ferrari N.V. (Italien)
    "RI.PA",     # Pernod Ricard S.A. (Frankreich)
    "SAF.PA",    # Safran S.A. (Frankreich)
    "SAN.PA",    # Sanofi S.A. (Frankreich)
    "SAP.DE",    # SAP SE (Deutschland)
    "SBMO.AS",   # SBM Offshore N.V. (Niederlande) - Achtung, nicht immer im Index
    "SGO.PA",    # Compagnie de Saint-Gobain S.A. (Frankreich)
    "SIE.DE",    # Siemens AG (Deutschland)
    "SLHN.SW",   # Swiss Life Holding AG (Schweiz) - je nach Index
    "SREN.SW",   # Swiss Re AG (Schweiz)
    "SU.PA",     # Schneider Electric S.E. (Frankreich)
    "TEF.MC",    # Telefonica S.A. (Spanien)
    "UCG.MI",    # UniCredit S.p.A. (Italien)
    "UL.PA",     # Unibail-Rodamco-Westfield SE (Frankreich)
    "VIV.PA",    # Vivendi SE (Frankreich)
    "VOW3.DE",   # Volkswagen AG (Deutschland, Vorzugsaktie)
    "WKL.AS",    # Wolters Kluwer N.V. (Niederlande)
    "ZURN.SW"    # Zurich Insurance Group AG (Schweiz)
]

In [5]:
START_TIME = "2019-01-01"
END_TIME = "2024-12-31"

In [6]:
# GET INDEX STOCK

df_ticker = yf.download(ticker_index, start=START_TIME, end=END_TIME)
df_ticker.columns = [col[0] for col in df_ticker.columns]
df_ticker = df_ticker.reset_index(names="Date")
df_ticker = df_ticker.rename(columns={"Open":"Open_Price","Close":"Close_Price","Volume":"Volume_traded"})
df_ticker = df_ticker.assign(Stock_ID = ticker_index)


df_ticker = df_ticker[["Stock_ID","Date","Open_Price","Close_Price","Low","High","Volume_traded"]]

[*********************100%***********************]  1 of 1 completed


In [7]:
# GET COMPANIES STOCK

for ticker in euro_stoxx_50_tickers:
    data = yf.download(ticker, start=START_TIME, end=END_TIME)
    data.columns = [col[0] for col in data.columns]
    data = data.reset_index(names="Date")
    data = data.rename(columns={"Open":"Open_Price","Close":"Close_Price","Volume":"Volume_traded"})
    data = data.assign(Stock_ID = ticker)
    
    data = data[["Stock_ID","Date","Open_Price","Close_Price","Low","High","Volume_traded"]]
    
    df_ticker = pd.concat([df_ticker, data], ignore_index=True, sort=False)
    
    
## FAILED ###
# 'CRG.IR'
# 'DAI.DE'
# 'DPW.DE'
# 'ENEI.MI'
# 'GRF.PA'
# 'INA.MI'
# 'NG.PA'

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['CRG.IR']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['DAI.DE']: YFTzMissingError('$%ticker%: possibly de

In [8]:
## WRITE DATA INTO Stock_Prices_Tbl

df_ticker.to_sql(
    name="Stock_Prices_Tbl",
    con=engine,
    schema=schema,
    if_exists="replace",
    index=False
)

261

In [9]:
## GET AND WRITE DATA INTO Country_Tbl

country = pd.read_csv("Country_tbl.csv", sep=";", encoding="utf-8")

country.to_sql(
    name="Country_Tbl",
    con=engine,
    schema=schema,
    if_exists="append",
    index=False
)

8

In [10]:
## GET AND WRITE DATA INTO Industry_Tbl

industry = pd.read_csv("Industry_tbl.csv", sep=";", encoding="utf-8")

industry.to_sql(
    name="Industry_Tbl",
    con=engine,
    schema=schema,
    if_exists="append",
    index=False
)

30

In [11]:
## GET AND WRITE DATA INTO Index_Tbl
data = [
    {
        "Index_ID":"^STOXX50E", 
        "Index_Name":"EuroStoxx 50"
    }
]


index = pd.DataFrame(data=data)

index.to_sql(
    name="Index_Tbl",
    con=engine,
    schema=schema,
    if_exists="append",
    index=False
)

1

In [12]:
## GET AND WRITE DATA INTO Company_Tbl

company = pd.read_csv("Company_tbl.csv", sep=";", encoding="utf-8")

company.to_sql(
    name="Company_Tbl",
    con=engine,
    schema=schema,
    if_exists="append",
    index=False
)


50